<img alt="jupyter" height="1080" src="./output.png" width="1420"/>

# 基本信息
1. 实验名称：实验一 pytorch基本操作与logistics、softmax回归的实现
2. 姓名：
3. 学号：
4. 日期：2022.10.2

---

# 一、任务1 Pytorch基本操作考察

## 1.1 任务内容

见实验要求

## 1.2 任务思路及代码
简单实现即可

In [1]:
import torch
import numpy as np

In [3]:
M = torch.rand(1, 3)
N = torch.rand(2, 1)
print("M", M)
print("N", N)
print("M - N", M - N)
print("M.subtract(N)", M.subtract(N))
print("torch.subtract(M, N)", torch.subtract(M, N))


M tensor([[0.1937, 0.8200, 0.7700]])
N tensor([[0.5907],
        [0.0518]])
M - N tensor([[-0.3970,  0.2292,  0.1793],
        [ 0.1420,  0.7682,  0.7183]])
M.subtract(N) tensor([[-0.3970,  0.2292,  0.1793],
        [ 0.1420,  0.7682,  0.7183]])
torch.subtract(M, N) tensor([[-0.3970,  0.2292,  0.1793],
        [ 0.1420,  0.7682,  0.7183]])


## 1.3 实验结果分析
第一种 调用tensor类的运算符重载直接计算
第二种 是类函数
第三种是torch的外部函数
运算过程中出现广播，最后都变成了 shape 2*3 矩阵减法

In [14]:
P = torch.normal(0, 0.01, size=(3, 2))
Q = torch.normal(0, 0.01, size=(4, 2))
print("P",P)
print('Q',Q)

P tensor([[ 0.0067,  0.0059],
        [-0.0033,  0.0239],
        [-0.0104, -0.0007]])
Q tensor([[-0.0091,  0.0119],
        [-0.0137,  0.0097],
        [ 0.0125,  0.0039],
        [-0.0045, -0.0084]])


In [15]:
print("the inverse of Q")
print(Q.T)

the inverse of Q
tensor([[-0.0091, -0.0137,  0.0125, -0.0045],
        [ 0.0119,  0.0097,  0.0039, -0.0084]])


In [17]:
print("P dot Q")
P@Q.T

P dot Q


tensor([[ 9.1661e-06, -3.4657e-05,  1.0619e-04, -7.9158e-05],
        [ 3.1417e-04,  2.7680e-04,  5.2596e-05, -1.8504e-04],
        [ 8.5815e-05,  1.3545e-04, -1.3219e-04,  5.2750e-05]])

In [21]:
x = torch.ones(1, requires_grad=True)
y_1 = x * x
y_2 = (x*x*x).detach()
y_3 = y_2 + y_1
y_3.backward()
x.grad

tensor([2.])

在 x^3 使用detach 相当于 对于 y3来说 y2是个常量 不求其导数，最后只有y2在反向传播计算图中被计算了。

# 二、任务2 实现logistics回归
## 2.1 从零开始实现logistics回归

In [1]:
# 从零开始实现logistics回归

import torch
import numpy as np

def mylogistics(X):
    z =  X.matmul(w.T) + b
    return torch.sigmoid(z)
    # result = torch.where(f>0.5, 1.0, 0.0)


loss = torch.nn.BCEWithLogitsLoss()

def mySGD(params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad
            param.grad.zero_()

x = torch.randint(-100,100, size=(100, 2), dtype=torch.float32)
label = []
for i in range(x.shape[0]):
    if x[i, 0] < x[i, 1]:
        label.append(1.0)
    else:
        label.append(0.0)

y = torch.tensor(label)
y = y.reshape((-1, 1))
print(x.shape)
print(y.shape)


torch.Size([100, 2])
torch.Size([100, 1])


In [2]:
def acc_logistics(y_hat, y):
    result = torch.where(y_hat>=0.5, 1, 0)
    return (result == y).type(y_hat.dtype).float().sum() / len(y)

In [19]:
y_hat_tem = torch.tensor([0.3, 0.9])
y_tem = torch.tensor([1, 0])
print(acc_logistics(y_hat_tem, y_tem).item())

0.0


In [22]:
def train_logistics_with_nothing(epochs, net, loss, lr, x, y, w, b):
    for epoch in range(epochs):
        y_hat = net(x)
        # print(y_hat)
        # print(y_hat.shape)
        acc = acc_logistics(y_hat, y)
        #  need a ()
        l = loss()(y_hat, y)
        l.backward()
        mySGD([w, b], lr)
        if ((epoch+1) % 100==0):
            print("epoch", epoch, " loss is", l.item(), " acc is", acc.item())

In [28]:
x = torch.randint(-100,100, size=(100, 2), dtype=torch.float32)
label = []
for i in range(x.shape[0]):
    if x[i, 0] < x[i, 1]:
        label.append(1.0)
    else:
        label.append(0.0)

w = torch.normal(0, 0.01, size=(1, x.shape[1]), dtype=torch.float32, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

y = torch.tensor(label)
y = y.reshape((-1, 1))
print(x.shape)
print(y.shape)

epochs = 1000
lr = 0.001
net = mylogistics

train_logistics_with_nothing(1000, mylogistics, torch.nn.BCELoss, lr, x, y, w, b)

torch.Size([100, 2])
torch.Size([100, 1])
epoch 99  loss is 0.07730719447135925  acc is 1.0
epoch 199  loss is 0.05772613361477852  acc is 1.0
epoch 299  loss is 0.047977838665246964  acc is 1.0
epoch 399  loss is 0.04174986481666565  acc is 1.0
epoch 499  loss is 0.037301160395145416  acc is 1.0
epoch 599  loss is 0.033910952508449554  acc is 1.0
epoch 699  loss is 0.031214550137519836  acc is 1.0
epoch 799  loss is 0.029003391042351723  acc is 1.0
epoch 899  loss is 0.02714785747230053  acc is 1.0
epoch 999  loss is 0.025562390685081482  acc is 1.0


## 2.2 利用torch.nn完成logistics回归

In [32]:
# 利用torch.nn完成logistics回归
# 这里利用torch.nn.Module 的继承写了个新类，其实只要有个nn.Linear层加个sigmod就好。甚至好像BCEwithLogistics函数自带一个sigmode层。

import torch
import torch.nn as nn

# 函数

class myLogisticsWithNN(nn.Module):
    def __inti__(self):
        super().__init__()

    def forward(self, X):
            z =  X.matmul(w.T) + b
            return torch.sigmoid(z)

def acc_logistics(y_hat, y):
    result = torch.where(y_hat>=0.5, 1, 0)
    return (result == y).type(y_hat.dtype).float().sum() / len(y)


def mySGD(params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad
            param.grad.zero_()


def train_logistics_with_nn(epochs, net, loss, lr, x, y, w, b):
    for epoch in range(epochs):
        y_hat = net().forward(x)
        # print(y_hat)
        # print(y_hat.shape)
        acc = acc_logistics(y_hat, y)
        #  need a ()
        l = loss()(y_hat, y)
        l.backward()
        mySGD([w, b], lr)
        if ((epoch+1) % 100==0):
            print("epoch", epoch, " loss is", l.item(), " acc is", acc.item())


In [33]:
# 数据随件初始化和训练

x = torch.randint(-100,100, size=(100, 2), dtype=torch.float32)
label = []
for i in range(x.shape[0]):
    if x[i, 0] < x[i, 1]:
        label.append(1.0)
    else:
        label.append(0.0)

w = torch.normal(0, 0.01, size=(1, x.shape[1]), dtype=torch.float32, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

y = torch.tensor(label)
y = y.reshape((-1, 1))
print(x.shape)
print(y.shape)

epochs = 1000
lr = 0.001
net = mylogistics

train_logistics_with_nn(epochs, myLogisticsWithNN, nn.BCELoss, lr, x, y, w, b)

torch.Size([100, 2])
torch.Size([100, 1])
epoch 99  loss is 0.053681716322898865  acc is 1.0
epoch 199  loss is 0.03993070498108864  acc is 1.0
epoch 299  loss is 0.0333995446562767  acc is 1.0
epoch 399  loss is 0.029321296140551567  acc is 1.0
epoch 499  loss is 0.026438962668180466  acc is 1.0
epoch 599  loss is 0.02425103262066841  acc is 1.0
epoch 699  loss is 0.022510986775159836  acc is 1.0
epoch 799  loss is 0.02108093537390232  acc is 1.0
epoch 899  loss is 0.01987663470208645  acc is 1.0
epoch 999  loss is 0.018843140453100204  acc is 1.0


# 三、任务3 实现softmax
## 3.1 从零开始实现softmax

In [ ]:
# 从零开始实现softmax
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
minst_train = torchvision.datasets.FashionMNIST(root="./Datasets/FashionMNIST", train=True, download=True, transform=transforms.ToTensor())
minst_test = torchvision.datasets.FashionMNIST(root="./Datasets/FashionMNIST", train=False, download=True, transform=transforms.ToTensor())

batch_size = 256
train_iter = DataLoader(minst_train, batch_size=batch_size, shuffle=True)
test_iter = DataLoader(minst_test, batch_size=batch_size, shuffle=False)


In [46]:
num_inputs = 784 #image of fashion minst is 28*28
num_outputs = 10 # and the num of class is 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [14]:
def softmax(X):
    X_exp = torch.exp(X)
    return X_exp / X_exp.sum(1, keepdim=True)

In [56]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [44]:
def cross_entropy(y_hat, y):
    return (- torch.log(y_hat[range(len(y_hat)), y])).mean()
# to gain a scalar
# for-loop is slow. we use a slice of y_hat to gain the all right class' prob, then log them.

In [17]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    result = y_hat.type(y.dtype) == y
    # op== is very sensitive with data type, so call tensor.type() to stay the same with y
    return float(result.type(y.dtype).sum())/len(y)

In [26]:
def mysgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [65]:
def eval_acc(net, data_iter):
    result = []
    with torch.no_grad():
        for X, y in data_iter:
            result.append(accuracy(net(X), y))
    return np.array(result).mean()

In [68]:

# def train_softmax(net, train_iter, loss, updater):
updater = torch.optim.SGD([W, b], lr=0.001)
epochs = 10
for epoch in range(epochs):
    for X, y in train_iter:
        # print(X.shape)
        # print(X)
        # print(y.shape)
        y_hat = net(X)
        # print(y_hat)
        # print(y)
        l = cross_entropy(y_hat, y)
        # print(l)
        updater.zero_grad()
        l.backward()
        updater.step()
    print("epoch", epoch, "train loss", l.item(), "train acc", accuracy(y_hat, y), "test_acc", eval_acc(net, test_iter))

epoch 2 train loss 0.35626840591430664 train acc 0.875 test_acc 0.8451171875
epoch 3 train loss 0.4013982117176056 train acc 0.8645833333333334 test_acc 0.844921875
epoch 4 train loss 0.4367528259754181 train acc 0.8541666666666666 test_acc 0.84443359375
epoch 5 train loss 0.36640170216560364 train acc 0.875 test_acc 0.84521484375
epoch 6 train loss 0.49508559703826904 train acc 0.8125 test_acc 0.8453125
epoch 7 train loss 0.3980194628238678 train acc 0.8958333333333334 test_acc 0.84443359375
epoch 8 train loss 0.32481905817985535 train acc 0.9166666666666666 test_acc 0.84521484375
epoch 9 train loss 0.4401278793811798 train acc 0.8333333333333334 test_acc 0.8443359375
epoch 0 train loss 0.3712468147277832 train acc 0.8645833333333334 test_acc 0.8453125
epoch 1 train loss 0.40290477871894836 train acc 0.875 test_acc 0.8453125
epoch 2 train loss 0.33752119541168213 train acc 0.875 test_acc 0.84521484375
epoch 3 train loss 0.45941439270973206 train acc 0.7916666666666666 test_acc 0.84511

## 3.2 利用torch.nn实现 softmax

In [85]:

# 这里参考了 李沐学ai的 3.7. softmax回归的简洁实现 里面的trick，即logsumexp技巧，在交叉熵里变相完成了softmax，这样避免上溢和下溢；而不是像上边一样利用nn.module实现一个新的class softmax。byt，李沐书中的这个方法真的太强了。

import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
minst_train = torchvision.datasets.FashionMNIST(root="./Datasets/FashionMNIST", train=True, download=True, transform=transforms.ToTensor())
minst_test = torchvision.datasets.FashionMNIST(root="./Datasets/FashionMNIST", train=False, download=True, transform=transforms.ToTensor())

batch_size = 256
train_iter = DataLoader(minst_train, batch_size=batch_size, shuffle=True)
test_iter = DataLoader(minst_test, batch_size=batch_size, shuffle=False)

loss1 = torch.nn.CrossEntropyLoss(reduction='none')
net1 = torch.nn.Sequential(torch.nn.Flatten(), torch.nn.Linear(784, 10))
updater1 = torch.optim.SGD(net1.parameters(), lr=0.1)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net1.apply(init_weights)

epochs = 10
def train_softmax_with_nn(epoch):
    for epoch in range(epochs):
        for X, y in train_iter:
            y_hat = net1(X)
            # print(y_hat)
            l = loss1(y_hat, y)
            updater1.zero_grad()
            l.mean().backward()
            updater1.step()
        print("epoch", epoch, "train loss", l.mean().item(), "train acc", accuracy(y_hat, y), "test_acc", eval_acc(net, test_iter))

In [86]:
train_softmax_with_nn(epochs)

epoch 0 train loss 0.6546421647071838 train acc 0.78125 test_acc 0.8451171875
epoch 1 train loss 0.5154418349266052 train acc 0.8541666666666666 test_acc 0.8451171875
epoch 2 train loss 0.552357017993927 train acc 0.8020833333333334 test_acc 0.8451171875
epoch 3 train loss 0.5014792084693909 train acc 0.8229166666666666 test_acc 0.8451171875
epoch 4 train loss 0.6346695423126221 train acc 0.8020833333333334 test_acc 0.8451171875
epoch 5 train loss 0.5494449734687805 train acc 0.8020833333333334 test_acc 0.8451171875
epoch 6 train loss 0.6015473008155823 train acc 0.7708333333333334 test_acc 0.8451171875
epoch 7 train loss 0.4758111536502838 train acc 0.8333333333333334 test_acc 0.8451171875
epoch 8 train loss 0.29127898812294006 train acc 0.90625 test_acc 0.8451171875
epoch 9 train loss 0.41524577140808105 train acc 0.875 test_acc 0.8451171875


# A1 实验心得
1. 在重名传递时，有时应该传递函数地址，有时又应该新建一个对象，直接传递，这里十分容易出错。比如在BCEloss中，一开始就是简单传递地址，结果爆出 不止一个可以比较的对象 之类的错误，由于经验不足简直无法入手。最后幸好，在同学的帮助下定位到了错误。
2. 一开始，从零开始实现logistics回归用了BCEwithLogistics损失函数，发现loss一直不降，但是acc接近1.0。查询文档后知道BCELogistics自带一个sigmod函数，可以直接配合Linear函数使用。遂换成了BCEloss函数。
3. softmax回归中通过参数类型转换很好的实现了acc函数。由bool转为float 直接记出正确的分类。
# A2 参考文献
> 动手学深度学习 https://zh-v2.d2l.ai/